Visualizing convnet filters

https://github.com/himanshurawlani/convnet-interpretability-keras/blob/master/Visualizing%20filters/visualizing_convnet_filters.ipynb

- for the gradients and : https://www.sicara.ai/blog/2019-08-28-interpretability-deep-learning-tensorflow
https://gist.github.com/RaphaelMeudec/31b7bba0b972ec6ec80ed131a59c5b3f#file-kernel_visualization-py

- for building blocks instead of layers (better visualization) as blocks (conv + pooling)
together can capture structures: https://github.com/nikhilroxtomar/Custom-Blocks-in-TensorFlow-using-Keras-API/blob/main/cifar10.py


In [139]:
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

In [140]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [141]:
from tensorflow import keras

class ConvBlock(keras.Model):
    def __init__(self, num_filters, kernel_size=(3, 3)):
        super(ConvBlock, self).__init__()

        self.conv = layers.Conv2D(num_filters, kernel_size)
        self.bn = layers.BatchNormalization()
        self.relu = layers.Activation("relu")
        self.pooling = layers.MaxPool2D((2, 2))

    def call(self, x, pool=True):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)

        if pool == True:
            x = self.pooling(x)

        return x

In [142]:
base_model = keras.applications.VGG19(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(50, 50, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [143]:
def CNN_blocks(input_shape=(50, 50, 3), output_class_count=2):
            
    inputs = layers.Input(shape=input_shape,name='Input')

    x = base_model.get_layer('block1_conv1')(inputs)
    x.trainable=False

    x = base_model.get_layer('block1_conv2')(x)
    x.trainable=False


    x = ConvBlock(6, kernel_size=(5, 5))(x, pool=False)

# layer 2   
    x= ConvBlock(16, kernel_size=(5, 5))(x,pool=False)
    

# layer 3
    x = ConvBlock(120, kernel_size=(5, 5))(x, pool=False)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(84, activation='relu',name='F6')(x)
    outputs = layers.Dense(units=output_class_count,activation='softmax',name='Output')(x)

    model = keras.Model(inputs, outputs)
    return model

In [144]:
#model = load_model('models/CNN.h5', custom_objects={'f1_score':f1_score})
model = CNN_blocks((50, 50, 3), 2)
model.load_weights('weights/CNN_weights.h5')

AttributeError: 'list' object has no attribute 'Input'

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    if 'conv' in layer.name:
        print(layer.name)
        print(len(layer.get_weights()))

Setting visualization variables

In [ ]:
def plotFilters(conv_filter):
    fig, axes = plt.subplots(1, 3, figsize=(5,5))
    axes = axes.flatten()
    for img, ax in zip( conv_filter, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

for layer in model.layers:
    if 'conv' in layer.name:
        filters, bias = layer.get_weights()
        print(layer.name, filters.shape)
         #normalize filter values between  0 and 1 for visualization
        f_min, f_max = filters.min(), filters.max()
        filters = (filters - f_min) / (f_max - f_min)  
        print(filters.shape[3])
        axis_x=1
        #plotting all the filters
        #for i in range(filters.shape[3]):
        for i in range(6):
            #get the filters
            filt=filters[:,:,:, i]
            plotFilters(filt)

Maximize the activation of a specific filter

In [ ]:
import numpy as np
import tensorflow as tf

# Layer name to inspect
layer_name = 'block1_conv2'

epochs = 100
step_size = 1.
filter_index = 1

# Create a connection between the input and the target layer
submodel = tf.keras.models.Model([model.inputs[0]], [model.get_layer(layer_name).output])

# Initiate random noise
input_img_data = np.random.random((1, 50, 50, 3))
input_img_data = (input_img_data - 0.5) * 20 + 128.

# Cast random noise from np.float64 to tf.float32 Variable
input_img_data = tf.Variable(tf.cast(input_img_data, tf.float32))

# Iterate gradient ascents
for _ in range(epochs):
    with tf.GradientTape() as tape:
        outputs = submodel(input_img_data)
        loss_value = tf.reduce_mean(outputs[:, :, :, filter_index])
    grads = tape.gradient(loss_value, input_img_data)
    normalized_grads = grads / (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
    input_img_data.assign_add(normalized_grads * step_size)

In [ ]:
img = input_img_data.numpy().astype(np.uint8)
img = img.squeeze()
img = img / 255
img.max()
img.shape

In [ ]:
plt.imshow(img, cmap='viridis')
plt.show()

In [ ]:
#dimensions of the generated pictures for each filter.
img_width = 50
img_height = 50

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
#layer_dict = dict([(layer.name, layer) for layer in model.layers[0:]])
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
layer_dict

In [ ]:
from keras import backend as K

# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [ ]:
def gradient_ascent(iterate):
    # step size for gradient ascent    
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

#         print('------>Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break
        
    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))

In [ ]:
def build_nth_filter_loss(filter_index, layer_name):
    """
    We build a loss function that maximizes the activation
    of the nth filter of the layer considered
    """
    
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])
    # Initiate random noise
    # Create a connection between the input and the target layer
    
    submodel = tf.keras.models.Model([model.inputs[0]], [model.get_layer(layer_name).output])

# Initiate random noise

    input_img_data = np.random.random((1, 50, 50, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128.

    # Cast random noise from np.float64 to tf.float32 Variable
    input_img_data = tf.Variable(tf.cast(input_img_data, tf.float32))

    for _ in range(epochs):
        with tf.GradientTape() as tape:
            outputs = submodel(input_img_data)
            loss_value = tf.reduce_mean(outputs[:, :, :, filter_index])
        grads = tape.gradient(loss_value, input_img_data)
        normalized_grads = grads / (tf.sqrt(tf.reduce_mean(tf.square(grads))) + 1e-5)
        input_img_data.assign_add(normalized_grads * step_size)

    # this function returns the loss and grads given the input picture
    #iterate = K.function([input_img], [loss_value, grads])

    if loss_value > 0:
        img = input_img_data.numpy().astype(np.float64)
        img = img.squeeze()
        img = deprocess_image(img)
        kept_filters.append((img, loss_value))
    #return iterate

In [ ]:
layers = [layer.name for layer in model.layers]

In [ ]:
layer = model.get_layer('conv_block_1')
range(min(layer.output.shape[-1], 100))
layer_name

In [ ]:
filter_index = 5
build_nth_filter_loss(filter_index, layer_name)

In [ ]:
import time

kept_filters = []
filters_dict = dict()
for layer_name in layers:
    if 'conv' in layer_name:
        layer = model.get_layer(layer_name)
        print('Processing filter for layer:', layer_name)
        for filter_index in range(min(layer.output.shape[-1], 100)):
            # print('Processing filter %d' % filter_index)

            start_time = time.time()
            build_nth_filter_loss(filter_index, layer_name)
            end_time = time.time()

    #         print('--->Filter %d processed in %ds' % (filter_index, end_time - start_time))
        filters_dict[layer.name] = kept_filters
        kept_filters = []

In [ ]:
for layer_name, kept_filters in filters_dict.items():
    print(layer_name, len(kept_filters))

In [ ]:
from keras.preprocessing.image import save_img

def stich_filters(kept_filters, layer_name):
    # By default, we will stich the best 64 (n*n) filters on a 8 x 8 grid.
    n = int(np.sqrt(len(kept_filters)))
    # the filters that have the highest loss are assumed to be better-looking.
    # we will only keep the top 64 filters.
    kept_filters.sort(key=lambda x: x[1], reverse=True)
    kept_filters = kept_filters[:n * n]

    # build a black picture with enough space for
    # our 8 x 8 filters of size 128 x 128, with a 5px margin in between
    margin = 5
    width = n * img_width + (n - 1) * margin
    height = n * img_height + (n - 1) * margin
    stitched_filters = np.zeros((width, height, 3))

    # fill the picture with our saved filters
    for i in range(n):
        for j in range(n):
            img, loss = kept_filters[i * n + j]
            width_margin = (img_width + margin) * i
            height_margin = (img_height + margin) * j
            stitched_filters[
                width_margin: width_margin + img_width,
                height_margin: height_margin + img_height, :] = img

    # save the result to disk
    save_img('img/stitched_filters_{}.png'.format(layer_name), stitched_filters)
    
for layer_name, kept_filters in filters_dict.items():
    print('Stiching filters for {}'.format(layer_name))
    stich_filters(kept_filters, layer_name)
    print('number of filters kept:', len(kept_filters))
    print('Completed.')

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

filter_name = 'conv_block_2'

img = image.img_to_array(image.load_img('img/stitched_filters_{}.png'.format(filter_name))) /255.
plt.figure(figsize=(17,17))
plt.imshow(img)
plt.title(filter_name)
plt.grid(False)